In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
#import mysql_conn

### Import Data

In [2]:
# Read in Apple store data
apple = pd.read_csv('Raw_Data/Apple/AppleStore.csv')
apple.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


In [3]:
# Read in Play Store data
google = pd.read_csv('Raw_Data/Google/googleplaystore.csv')
google.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


### Rename Columns

In [4]:
# Rename Apple columns
a_cols = ['track_name', 'id', 'size_bytes', 'price', 'user_rating', 'cont_rating', 'prime_genre']
apple = apple[a_cols]
apple = apple.rename(columns = {'track_name':'a_name', 
                                'id':'a_id',
                                'size_bytes':'a_size_bytes', 
                                'price':'a_price', 
                                'user_rating':'a_user_rating', 
                                'cont_rating':'a_content_rating', 
                                'prime_genre':'a_category'})
apple.head()

,a_name,a_id,a_size_bytes,a_price,a_user_rating,a_content_rating,a_category
0,PAC-MAN Premium,281656475,100788224,3.99,4.0,4+,Games
1,Evernote - stay organized,281796108,158578688,0.00,4.0,4+,Productivity
2,"WeatherBug - Local Weather, Radar, Maps, Alerts",281940292,100524032,0.00,3.5,4+,Weather
3,"eBay: Best App to Buy, Sell, Save! Online Shop...",282614216,128512000,0.00,4.0,12+,Shopping
4,Bible,282935706,92774400,0.00,4.5,4+,Reference


In [5]:
#google.loc[google.g_name.str.contains('eBay')]

In [6]:
# Rename Google columns
g_cols = ['App', 'Size', 'Price', 'Rating', 'Content Rating', 'Category']
google = google[g_cols]
google = google.rename(columns = {'App':'g_name', 
                        'Size':'g_size_mb', 
                        'Price':'g_price', 
                        'Rating':'g_user_rating', 
                        'Content Rating':'g_content_rating', 
                        'Category':'g_category'})
google.head()

,g_name,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
0,Photo Editor & Candy Camera & Grid & ScrapBook,19M,0,4.1,Everyone,ART_AND_DESIGN
1,Coloring book moana,14M,0,3.9,Everyone,ART_AND_DESIGN
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",8.7M,0,4.7,Everyone,ART_AND_DESIGN
3,Sketch - Draw & Paint,25M,0,4.5,Teen,ART_AND_DESIGN
4,Pixel Draw - Number Art Coloring Book,2.8M,0,4.3,Everyone,ART_AND_DESIGN


### Remove Apps that do not appear on both datasets

In [7]:
# Drop duplicates
google = google.drop_duplicates(subset = 'g_name')
apple = apple.drop_duplicates(subset = 'a_name')

In [8]:
apple_merge = apple.rename(columns = {'a_name':'name'})
google_merge = google.rename(columns = {'g_name':'name'})

In [9]:
# Remove apps that do not appear in both datasets
apps_merge = pd.merge(apple_merge, google_merge, on = 'name')

In [10]:
apps_merge.head()

,name,a_id,a_size_bytes,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
0,Bible,282935706,92774400,0.0,4.5,4+,Reference,Varies with device,0,4.7,Teen,BOOKS_AND_REFERENCE
1,Facebook,284882215,389879808,0.0,3.5,4+,Social Networking,Varies with device,0,4.1,Teen,SOCIAL
2,LinkedIn,288429040,273844224,0.0,3.5,4+,Social Networking,Varies with device,0,4.2,Everyone,SOCIAL
3,Google Earth,293622097,37214208,0.0,3.5,4+,Travel,Varies with device,0,4.3,Everyone,TRAVEL_AND_LOCAL
4,PAC-MAN,293778748,100849664,0.0,3.0,4+,Games,37M,0,4.2,Everyone,GAME


In [11]:
common_apps = list(apps_merge.name)

# Column Formatting

### Format app size columns

In [12]:
#print(apple.a_size_mb.unique())
#print(google.g_size_mb.unique())
apps_merge.g_size_mb.unique()

array(['Varies with device', '37M', '46M', '63M', '33M', '34M', '19M',
       '61M', '35M', '31M', '30M', '20M', '8.3M', '23M', '32M', '36M',
       '18M', '2.3M', '43M', '41M', '22M', '44M', '42M', '8.7M', '15M',
       '1.9M', '6.6M', '67M', '90M', '29M', '69M', '80M', '3.9M', '96M',
       '12M', '48M', '98M', '85M', '16M', '17M', '73M', '21M', '49M',
       '25M', '62M', '94M', '76M', '7.8M', '50M', '11M', '14M', '66M',
       '45M', '100M', '24M', '53M', '68M', '59M', '74M', '71M', '9.5M',
       '5.8M', '75M', '79M', '70M', '58M', '4.2M', '92M', '82M', '95M',
       '7.4M', '57M', '88M', '51M', '26M', '6.9M', '56M', '28M', '99M',
       '6.5M', '7.6M', '72M', '40M', '52M', '78M', '39M', '83M', '7.9M',
       '54M', '27M', '97M', '4.0M', '93M', '64M', '4.9M', '65M', '55M'],
      dtype=object)

In [13]:
# Convert Apple app size from bytes to megabytes
a_size_mb = []

for byte in list(apps_merge.a_size_bytes):
    megabyte = byte / 1024 / 1024
    a_size_mb.append(megabyte)

apps_merge.a_size_bytes = a_size_mb
apps_merge = apps_merge.rename(columns = {'a_size_bytes':'a_size_mb'})

# Round decimal places
apps_merge.a_size_mb = apps_merge.a_size_mb.round(1)

In [14]:
# Remove 'M' from Google app sizes
apps_merge.g_size_mb = apps_merge.g_size_mb.str[:-1]

In [15]:
# Remove rows where google size 'varies with device'
apps_merge = apps_merge.loc[apps_merge['g_size_mb'] != 'Varies with devic']


In [16]:
apps_merge.head()

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
4,PAC-MAN,293778748,96.2,0.0,3.0,4+,Games,37,0,4.2,Everyone,GAME
7,Citi Mobile®,301724680,268.2,0.0,3.5,4+,Finance,46,0,4.0,Everyone,FINANCE
9,DIRECTV,307386350,144.4,0.0,3.5,4+,Entertainment,63,0,4.1,Teen,FAMILY
15,Wells Fargo Mobile,311548709,54.7,0.0,3.0,4+,Finance,37,0,4.4,Everyone,FINANCE
16,USAA Mobile,312325565,156.8,0.0,4.0,4+,Finance,33,0,4.5,Everyone,FINANCE


### Format price columns

In [17]:
print(apps_merge.a_price.unique())
print(apps_merge.g_price.unique())

[ 0.    1.99  4.99  0.99 19.99  9.99  2.99 14.99  6.99  3.99  7.99  5.99]
['0' '$0.99' '$4.99' '$1.99' '$24.99' '$14.99' '$9.99' '$2.99' '$1.20'
 '$7.99' '$6.99' '$3.99' '$5.99']


In [18]:
apps_merge.loc[apps_merge.g_price.str.contains('Everyone')]

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category


In [19]:
# Drop rows with errors
apps_merge = apps_merge[~apps_merge.g_price.str.contains("Everyone")]

# Format data to match Apple dataset
apps_merge['g_price'] = apps_merge.g_price.str.lstrip('$')


(apps_merge.g_price.unique())

array(['0', '0.99', '4.99', '1.99', '24.99', '14.99', '9.99', '2.99',
       '1.20', '7.99', '6.99', '3.99', '5.99'], dtype=object)

In [20]:
apps_merge.head()

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
4,PAC-MAN,293778748,96.2,0.0,3.0,4+,Games,37,0,4.2,Everyone,GAME
7,Citi Mobile®,301724680,268.2,0.0,3.5,4+,Finance,46,0,4.0,Everyone,FINANCE
9,DIRECTV,307386350,144.4,0.0,3.5,4+,Entertainment,63,0,4.1,Teen,FAMILY
15,Wells Fargo Mobile,311548709,54.7,0.0,3.0,4+,Finance,37,0,4.4,Everyone,FINANCE
16,USAA Mobile,312325565,156.8,0.0,4.0,4+,Finance,33,0,4.5,Everyone,FINANCE


In [21]:
apps_merge.head()

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
4,PAC-MAN,293778748,96.2,0.0,3.0,4+,Games,37,0,4.2,Everyone,GAME
7,Citi Mobile®,301724680,268.2,0.0,3.5,4+,Finance,46,0,4.0,Everyone,FINANCE
9,DIRECTV,307386350,144.4,0.0,3.5,4+,Entertainment,63,0,4.1,Teen,FAMILY
15,Wells Fargo Mobile,311548709,54.7,0.0,3.0,4+,Finance,37,0,4.4,Everyone,FINANCE
16,USAA Mobile,312325565,156.8,0.0,4.0,4+,Finance,33,0,4.5,Everyone,FINANCE


### Format content rating

In [22]:
apps_merge.g_content_rating.unique()

array(['Everyone', 'Teen', 'Everyone 10+', 'Mature 17+'], dtype=object)

In [23]:
apps_merge.a_content_rating.unique()

array(['4+', '12+', '17+', '9+'], dtype=object)

In [24]:
# Create new DataFrame to identify equivalent content ratings
content_ratings = pd.DataFrame(data = {'id':[1, 2, 3, 4],  
                     'apple':['4+', '9+', '12+', '17+'], 
                     'google':['Everyone', 'Everyone 10+', 'Teen', 'Mature 17+']})
content_ratings = content_ratings.set_index('id')
content_ratings

,apple,google
id,,
1,4+,Everyone
2,9+,Everyone 10+
3,12+,Teen
4,17+,Mature 17+


In [25]:
# Rename content ratings to match

# Apple
apps_merge['a_content_rating'] = apps_merge.a_content_rating.replace('4+', '1')
apps_merge['a_content_rating'] = apps_merge.a_content_rating.replace('9+', '2')
apps_merge['a_content_rating'] = apps_merge.a_content_rating.replace('12+', '3')
apps_merge['a_content_rating'] = apps_merge.a_content_rating.replace('17+', '4')

# Google
apps_merge['g_content_rating'] = apps_merge.g_content_rating.replace('Everyone', '1')
apps_merge['g_content_rating'] = apps_merge.g_content_rating.replace('Everyone 10+', '2')
apps_merge['g_content_rating'] = apps_merge.g_content_rating.replace('Teen', '3')
apps_merge['g_content_rating'] = apps_merge.g_content_rating.replace('Mature 17+', '4')

In [26]:
apps_merge.head()

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
4,PAC-MAN,293778748,96.2,0.0,3.0,1,Games,37,0,4.2,1,GAME
7,Citi Mobile®,301724680,268.2,0.0,3.5,1,Finance,46,0,4.0,1,FINANCE
9,DIRECTV,307386350,144.4,0.0,3.5,1,Entertainment,63,0,4.1,3,FAMILY
15,Wells Fargo Mobile,311548709,54.7,0.0,3.0,1,Finance,37,0,4.4,1,FINANCE
16,USAA Mobile,312325565,156.8,0.0,4.0,1,Finance,33,0,4.5,1,FINANCE


### Categories

In [27]:
apps_merge.a_category.unique()

array(['Games', 'Finance', 'Entertainment', 'Shopping', 'News',
       'Productivity', 'Food & Drink', 'Sports', 'Business', 'Travel',
       'Education', 'Social Networking', 'Medical', 'Weather',
       'Photo & Video', 'Lifestyle', 'Book', 'Catalogs', 'Utilities',
       'Health & Fitness'], dtype=object)

In [28]:
apps_merge.g_category.unique()

array(['GAME', 'FINANCE', 'FAMILY', 'SHOPPING', 'NEWS_AND_MAGAZINES',
       'PRODUCTIVITY', 'FOOD_AND_DRINK', 'SPORTS', 'BUSINESS',
       'ENTERTAINMENT', 'MAPS_AND_NAVIGATION', 'TRAVEL_AND_LOCAL',
       'EDUCATION', 'SOCIAL', 'MEDICAL', 'LIFESTYLE', 'WEATHER',
       'PHOTOGRAPHY', 'VIDEO_PLAYERS', 'COMMUNICATION',
       'HEALTH_AND_FITNESS', 'ART_AND_DESIGN'], dtype=object)

In [29]:
apps_merge.a_category.unique()

array(['Games', 'Finance', 'Entertainment', 'Shopping', 'News',
       'Productivity', 'Food & Drink', 'Sports', 'Business', 'Travel',
       'Education', 'Social Networking', 'Medical', 'Weather',
       'Photo & Video', 'Lifestyle', 'Book', 'Catalogs', 'Utilities',
       'Health & Fitness'], dtype=object)

In [30]:
#categories = pd.DataFrame(data = {'id':list(range(1, 23)),  
#                     'apple':['Reference', 'Social Networking', 'Travel', 'Games',
#                              'Food & Drink', 'Utilities', 'Finance', 'Entertainment',
#                              'Business', 'Shopping', 'News', 'Photo & Video', 'Productivity',
#                              'Navigation', 'Sports', 'Education', 'Health & Fitness',
#                              'Lifestyle', 'Medical', 'Weather', 'Book', 'Catalogs'], 
#                     'google':['BOOKS_AND_REFERENCE', 'SOCIAL', 'TRAVEL_AND_LOCAL', 'GAME',
#                               'FOOD_AND_DRINK', 'TOOLS', 'FINANCE', 'FAMILY', 'ENTERTAINMENT',
#                               'PRODUCTIVITY', 'BUSINESS', 'COMMUNICATION', 'SHOPPING',
#                               'NEWS_AND_MAGAZINES', 'HEALTH_AND_FITNESS', 'SPORTS',
#                               'MAPS_AND_NAVIGATION', 'EDUCATION', 'LIFESTYLE', 'PHOTOGRAPHY',
#                               'MEDICAL', 'WEATHER', 'VIDEO_PLAYERS', 'ART_AND_DESIGN']})
#categories = content_ratings.set_index('id')
#categories

In [31]:
apps_merge.a_category.unique()

array(['Games', 'Finance', 'Entertainment', 'Shopping', 'News',
       'Productivity', 'Food & Drink', 'Sports', 'Business', 'Travel',
       'Education', 'Social Networking', 'Medical', 'Weather',
       'Photo & Video', 'Lifestyle', 'Book', 'Catalogs', 'Utilities',
       'Health & Fitness'], dtype=object)

In [32]:
apps_merge.loc[apps_merge['a_category'] == 'Travel']

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
29,DB Navigator,343555245,80.9,0.0,3.5,1,Travel,20,0,4.0,1,MAPS_AND_NAVIGATION
30,Southwest Airlines,344542975,84.7,0.0,3.0,1,Travel,8.3,0,3.9,1,TRAVEL_AND_LOCAL
46,Fly Delta,388491656,134.3,0.0,3.0,1,Travel,46,0,3.7,1,TRAVEL_AND_LOCAL
78,United Airlines,449945214,134.4,0.0,2.5,1,Travel,80,0,3.5,1,TRAVEL_AND_LOCAL


In [33]:
apps_merge.loc[apps_merge['g_category'] == 'TRAVEL_AND_LOCAL']

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
30,Southwest Airlines,344542975,84.7,0.0,3.0,1,Travel,8.3,0,3.9,1,TRAVEL_AND_LOCAL
46,Fly Delta,388491656,134.3,0.0,3.0,1,Travel,46,0,3.7,1,TRAVEL_AND_LOCAL
78,United Airlines,449945214,134.4,0.0,2.5,1,Travel,80,0,3.5,1,TRAVEL_AND_LOCAL


In [34]:
#apps_merge.loc[~(apps_merge['a_category'] == apps_merge['g_category'])]

In [35]:
# Reset numeric values of index
apps_merge = apps_merge.reset_index(drop = True)
apps_merge

,name,a_id,a_size_mb,a_price,a_user_rating,a_content_rating,a_category,g_size_mb,g_price,g_user_rating,g_content_rating,g_category
0,PAC-MAN,293778748,96.2,0.00,3.0,1,Games,37,0,4.2,1,GAME
1,Citi Mobile®,301724680,268.2,0.00,3.5,1,Finance,46,0,4.0,1,FINANCE
2,DIRECTV,307386350,144.4,0.00,3.5,1,Entertainment,63,0,4.1,3,FAMILY
3,Wells Fargo Mobile,311548709,54.7,0.00,3.0,1,Finance,37,0,4.4,1,FINANCE
4,USAA Mobile,312325565,156.8,0.00,4.0,1,Finance,33,0,4.5,1,FINANCE
5,Best Buy,314855255,172.1,0.00,4.0,1,Shopping,34,0,4.4,1,SHOPPING
6,Sky News,316391924,87.4,0.00,3.0,3,News,19,0,4.0,2,NEWS_AND_MAGAZINES
7,Mad Libs,326885152,113.0,0.00,3.0,1,Entertainment,61,0,4.0,1,FAMILY
8,Dropbox,327630330,172.4,0.00,3.5,1,Productivity,61,0,4.4,1,PRODUCTIVITY
9,Starbucks,331177714,128.8,0.00,4.5,1,Food & Drink,35,0,4.5,1,FOOD_AND_DRINK


# Create connection to database

In [51]:
connection_string = (
    f"root:eAVuM6bo4LgR@localhost/apps_db")
engine = create_engine(f'mysql://{connection_string}?charset=utf8')
#engine = create_engine('sqlite://', echo=False)

In [52]:
engine.table_names()

['app_store_descriptions', 'apple_store_descriptions', 'apps']

In [53]:
apps_merge.to_sql(
    name='apps', con=engine, 
    if_exists='replace', index=True)

In [65]:
name_desc = pd.read_sql('SELECT apps.name, app_store_descriptions.app_desc FROM app_store_descriptions INNER JOIN apps ON apps.a_id = app_store_descriptions.id', con = engine)

In [67]:
type(name_desc)

pandas.core.frame.DataFrame

In [69]:
name_desc.to_csv('name_desc.csv')